# Módule 6: Ensamblajes *De novo*

## Descripción general

Después de generar lecturas de secuenciación y llevar a cabo una evaluación de calidad, el siguiente paso es determinar cómo encajan las lecturas buscando superposiciones entre ellas; este proceso se denomina ensamblaje del genoma. Aquí puede ver los pasos generales de un flujo de trabajo de ensamblaje del genoma.*Los pasos 1 y 2 ya se han completado.*


![steps](images/ass.jpg)

*Tomado de: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5850084/*

Los datos de secuenciación por Illumina vienen como tramos relativamente cortos (35 - 150 pares de bases) de ADN, alrededor de 6 mil millones de ellos. Estas secuencias individuales se denominan lecturas de secuenciación. Hay una variedad de programas de ensamblaje que se han diseñado específicamente para ensamblar genomas a partir de datos de lectura corta de secuencias. El ensamblaje del genoma utilizando lecturas cortas (~100 pb) es complicado debido a la alta frecuencia de repeticiones más largas que la longitud de lectura de secuencia en los genomas, por ejemplo: elementos IS, operones de ARNr; y la gran cantidad de datos que los programas tienen que manejar. Además de encontrar superposiciones en la secuencia, los programas de ensamblaje también pueden usar información del tamaño de inserción predicho donde se usan lecturas emparejadas, para vincular y posicionar lecturas en un ensamblaje.

Cuando un genoma se ensambla sin ninguna secuencia de referencia con la que compararlo o contra el que construir un andamiaje, se denomina ensamblaje *de novo*. El ensamblaje *de novo* puede no producir genomas completos, sino que se fragmentará en múltiples secuencias contiguas (contigs), cuyo orden es arbitrario y no guarda necesariamente ninguna relación con su orden real en el genoma.

Cuando se dispone de una secuencia de referencia estrechamente relacionada, es posible mejorar un ensamblaje ordenando los contigs en comparación con la referencia y también transfiriendo la anotación. En este caso, estará presente casi todo el genoma, es decir, genes y características, pero habrá algunas regiones que contendrán lagunas o contigs que no se colocarán con precisión porque no están presentes en la referencia utilizada. Aunque técnicamente incompletos, los ensamblajes ordenados del genoma pueden proporcionar información valiosa sobre la genética y la biología de un organismo.

![denovo](images/denovo.jpg)

*Tomado de: https://doi.org/10.1038/nmeth.1935*


### Instalar condacolab

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

### Instalar programas

In [ ]:
# Instalar SPAdes y QUAST
!conda install bioconda::spades
!conda install bioconda::quast


In [ ]:
# Compruebe si SPAdes se instaló correctamente
!spades.py --help

In [ ]:
# Compruebe si se QUAST se instaló correctamente
!quast.py --help

### Descargar datos

In [ ]:
!wget

## Parte 1: Generación de un ensamblaje *de novo* para una sola cepa

En esta sección, usaremos [SPAdes](https://github.com/ablab/spades), que es uno de varios ensambladores de novo que usan conjuntos de lectura cortos como entrada (por ejemplo, Illumina Reads), y el método de ensamblaje se basa en gráficos de Bruijn. 

Reuniremos el genoma de la cepa 17150_4#79 de Streptococcus pneumoniae (número de acceso: ERR1065718).

Ejecute el comando en la terminal para ejecutar SPAdes:

In [ ]:
# Correr SPAdes
!spades.py -1 17150_4#79_1.fastq.gz -2 17150_4#79_2.fastq.gz --careful --cov-cutoff auto -o spades_assembly

Una explicación de este comando es la siguiente:

**spades.py**: es la herramienta

**-1**: indicador para el archivo de entrada de lecturas forward

**-2**: indicador para el archivo de entrada de lecturas reverse

-**-careful**: minimiza los desajustes y los indeles cortos

**--cov-cutoff auto**: calcula el umbral de cobertura (en lugar de la configuración predeterminada, "apagado")

**-o**: directorio de salida

Vaya al directorio de salida (spades_assembly) y mire el archivo contigs.fa:

In [ ]:
# Cambiar al directorio de salida
%cd spades_assembly

In [ ]:
# Consulte el informe

## Generación de un ensamblaje *de novo* para múltiples cepas

También podemos ensamblar genomas de múltiples cepas. Navegue a la carpeta multiple_files y siga los pasos a continuación.

### Paso 1: Primero crearemos una carpeta para cada par de archivos fastq comprimidos y el nombre de la identificación de la cepa usando el comando:

for x in *1.fastq.gz; do mkdir ${x%%_1.fastq.gz} ; mv $x ${x%%_1.fastq.gz}; mv ${x%%1.fastq.gz}2.fastq.gz ${x%%_1.fastq.gz}; done

Una explicación de este comando es la siguiente:

**for x in *1.fastq.gz**; Esto inicia un ciclo for que itera sobre todos los archivos en el directorio actual que terminan con "1.fastq.gz".

**do**  Esto inicia el bloque de código que se ejecutará para cada archivo.

**mkdir ${x%%_1.fastq.gz}** Esto crea un nuevo directorio con el mismo nombre que el archivo, pero con el sufijo "_1.fastq.gz" eliminado.

**mv $x ${x%%_1.fastq.gz}** Esto mueve el archivo original al nuevo directorio creado en el paso anterior.

**mv ${x%%1.fastq.gz}2.fastq.gz ${x%%_1.fastq.gz}** Esto mueve un segundo archivo que tiene el mismo prefijo que el primer archivo, pero con un "2.fastq sufijo .gz", en el nuevo directorio creado en el primer paso.

**done** Esto finaliza el bucle for.

En general, este script está diseñado para tomar datos de secuenciación pareados que se almacenan en dos archivos separados con nombres que terminan en **"_1.fastq.gz"** y **"_2.fastq.gz"**, y organizarlos en directorios según el prefijo de los nombres de archivo.

### Paso 2: Luego ejecutaremos SPAdes usando el comando:

In [ ]:
for x in *#* ; do docker_run staphb/spades spades.py --pe1-1 $x/${x}_1.fastq.gz --pe1-2 $x/${x}_2.fastq.gz --careful --cov-cutoff auto -o $x"_output"; done

Una explicación de este comando es la siguiente:

**for x in *#*** Esto inicia un ciclo que itera sobre todos los directorios en el directorio de trabajo actual que tienen un nombre que contiene el carácter "#". La variable de bucle x se establecerá en cada nombre de directorio a su vez.

**do** Esta palabra clave inicia el bloque de comandos que se ejecutará para cada iteración del bucle.

**spades.py** Este es el comando de ejecución SPAdes.

**--pe1-1 $x/${x}_1.fastq.gz** Esto especifica la ruta al primer archivo de lectura de extremo emparejado para SPAdes. La variable $x se usa para hacer referencia al nombre del directorio actual y se agrega ${x}_1.fastq.gz para crear el nombre completo del archivo.

**--pe1-2 $x/${x}_2.fastq.gz** Esto especifica la ruta al segundo archivo de lectura de extremo emparejado para SPAdes. Se usa la misma variable de nombre de directorio $x que la anterior, pero con ${x}_2.fastq.gz agregado para crear el nombre de archivo completo para el segundo archivo de lectura.

**--careful** Esto le dice a SPAdes que use un algoritmo más conservador para la corrección de errores.

**--cov-cutoff** auto Esto le dice a SPAdes que determine automáticamente el límite de cobertura para filtrar contigs de baja cobertura durante el ensamblaje.

**-o $x"_output"**  Esto especifica el directorio de salida para el ensamblado de SPAdes. El nombre del directorio se crea agregando "_output" al nombre del directorio actual $x.

**;** Este es el separador de comandos que le dice a Bash que ejecute el comando anterior antes de pasar al siguiente.

**done** Esta palabra clave marca el final del bloque de bucle.

En general, esta secuencia de comandos automatiza la ejecución del comando de ensamblaje SPAdes en varios conjuntos de lectura emparejados, cada uno contenido en un directorio con un nombre que contiene el carácter "#".

___

# Control calidad para genomas ensamblados

Las tecnologías modernas de secuenciación de ADN no pueden producir la secuencia completa de un cromosoma. En cambio, generan una gran cantidad de lecturas, que van desde docenas hasta miles de bases consecutivas, muestreadas de diferentes partes del genoma. Los software de ensamblaje combinan las lecturas en regiones más grandes llamadas contigs. Sin embargo, las tecnologías y softwares de secuenciación actuales enfrentan muchos desafíos que impiden la reconstrucción de cromosomas completos, incluyendo errores en las lecturas y grandes repeticiones en el genoma.

Diferentes programas ensambladores utilizan diferentes enfoques heurísticos para abordar estos desafíos, lo que da como resultado muchas diferencias en los contigs que generan. Esto lleva a las preguntas de cómo evaluar la calidad de un ensamblaje y cómo comparar diferentes ensamblajes.

Otras lecturas: https://academic.oup.com/bioinformatics/article/29/8/1072/228832

## Evaluación de ensamblajes generados

Usaremos la herramienta QUAST en esta sección.

[QUAST](https://github.com/ablab/quast) significa (QUality Assessment Tool) Herramienta de evaluación de calidad. Evalúa ensamblajes de genoma/metagenoma mediante el cálculo de varias métricas. El kit de herramientas QUAST actual incluye la herramienta QUAST general para ensamblajes de genomas, MetaQUAST, la extensión para conjuntos de datos metagenómicos, QUAST-LG, la extensión para genomas grandes (p. ej., mamíferos) e Icarus, el visualizador interactivo para estas herramientas.

El paquete QUAST funciona con y sin genomas de referencia. Sin embargo, es mucho más informativo si se proporciona al menos un genoma de referencia cercano junto con los ensamblajes. La herramienta acepta múltiples ensamblajes, por lo que es adecuada para la comparación.

Evaluaremos la calidad de los contigs "17150_4#79" generados a partir del ensamblaje De novo descrito en la página anterior. Copie el archivo contigs.fasta de la carpeta "spades_assembly" a la carpeta Section_tres.

Ejecutaremos la herramienta QUAST en estos contigs usando el comando:

In [ ]:
!quast.py contigs.fasta -r Reference_sequence_GPSC46.fa -g PROKKA_03052023.gff -1 17150_4#79_1.fastq.gz -2 17150_4#79_2.fastq.gz -o quast_17150_4#79_output

An explanation of this command is as follows:

**quast.py**: es la herramienta

**-r Reference_sequence_GPSC46.fa**: especifica la secuencia de referencia

**-g PROKKA_03052023.gff**: especifica el gen en el genoma de referencia (salida PROKKA de Reference_sequence_GPSC46.fa)

**-1 17150_4#79_1.fastq.gz**: archivo de entrada de lecturas directas

**-2 17150_4#79_2.fastq.gz**: archivo de entrada de lecturas inversas

**-o quast_ 17150_4#79_output**: especifica la carpeta de salida

Cuando QUAST esté completo, imprimirá el siguiente mensaje en la pantalla de su terminal:

Navegue a la carpeta de salida `"quast_ 17150_4#79_output"` usando el comando `%cd` y explore su contenido. Una descripción de los archivos de salida es la siguiente:

**report.txt**:  versión separada por tabuladores del resumen, adecuada para hojas de cálculo (Google Docs, Excel, etc.)

**report.tex**:  versión LaTeX del resumen

**icarus.html**: menú principal de Icarus con enlaces a visores interactivos.

**report.pdf**: todos los demás gráficos combinados con todas las tablas (el archivo se crea si está instalada la biblioteca python matplotlib)

**report.html**: versión HTML del informe con diagramas interactivos dentro

**contigs_reports/**: (solo si se proporciona un genoma de referencia)

**misassemblies_report**: informe detallado sobre desensamblajes

**unaligned_report**: informe detallado sobre contigs desalineados y parcialmente desalineados

**k_mer_stats**/: (solo si se especifica la opción --k-mer-stats)

**kmers_report**:  informe detallado sobre métricas basadas en k-mer

**reads_stats/**: (solo si se proporcionan lecturas)

**reads_report**: informe detallado sobre las estadísticas de lecturas asignadas.

## Interpretación

### Descripción de las métricas 

#### 1. Informe resumido: basado en el archivo report.txt en la carpeta de salida

**`#` contigs** es el número total de contigs en el ensamblaje.

**Largest contig** es la longitud del contig más largo del ensamblaje.

**Total length** es el número total de bases en el ensamblaje.

**Reference length** es el número total de bases en el genoma de referencia.

**GC (%)** es el número total de nucleótidos G y C en el ensamblaje, dividido por la longitud total del ensamblaje.

**Reference GC (%)**  es el porcentaje de nucleótidos G y C en el genoma de referencia.

**N50** es la longitud para la cual la colección de todos los contigs de esa longitud o más cubre al menos la mitad de un conjunto.

**NG50** es la longitud para la cual la colección de todos los cóntigos de esa longitud o más cubre al menos la mitad del genoma de referencia (esta métrica se calcula solo si se proporciona el genoma de referencia).

**Nx and NGx** (para x entre 0 y 100) ase definen de manera similar a N50 pero con x % en lugar de 50 %. El valor de x se establece con --x-for-Nx (90 por defecto).

**L50 (Lx, LG50, LGx)** es el número de contigs igual o mayor que N50 (Nx, NG50, NGx)

En otras palabras, L50, por ejemplo, es el número mínimo de contigs que cubren la mitad del conjunto.

**`#` misassemblies** es el número de posiciones en los contigs (puntos de interrupción) que satisfacen uno de los siguientes criterios:

- La secuencia flanqueante izquierda se alinea a más de 1 kbp de distancia de la secuencia flanqueante derecha en la referencia;
- Las secuencias flanqueantes se superponen en más de 1 kpb;
- Las secuencias flanqueantes se alinean con diferentes hebras o diferentes cromosomas;
- Las secuencias flanqueantes se alinean en diferentes genomas de referencia (solo MetaQUAST).

*Esta métrica requiere un genoma de referencia.*

**`#` misassembled contigs** es el número de contigs que contienen eventos de ensamblaje incorrecto 

**Misassembled contigs length**  es el número total de bases en contigs mal ensamblados.

**`#` local misassemblies** es el número de posiciones en los contigs (puntos de interrupción) que satisfacen las siguientes condiciones:

1. El espacio o superposición entre las secuencias flanqueantes izquierda y derecha es inferior a 1 kpb y superior a 200 pb (la longitud máxima de indel).
2. Las secuencias flanqueantes izquierda y derecha están ambas en la misma hebra del mismo cromosoma del genoma de referencia.

**`#` scaffold gap ext. mis.** es el número de posiciones en los andamios (puntos de interrupción) donde las secuencias laterales se combinan en el andamio en la distancia incorrecta (suficiente para informar un gran error de montaje).

**`#` structural variations** es el número de desensamblajes que coinciden con las variaciones estructurales del genoma (si se proporcionan lecturas o un archivo BEDPE con SV, consulte --reads1/reads2 y --sv-bedpe).

**`#` unaligned mis. contigs**  es el número de contigs que tienen un número de bases no alineadas superior al 50 % de la longitud del contig y al menos un evento de montaje incorrecto en su fragmento alineado. Tales contigs probablemente no estén relacionados con el genoma de referencia, por lo que sus desensamblajes pueden no ser errores reales sino diferencias entre el organismo ensamblado y el de referencia.

**`#` unaligned contigs** es el número de contigs que no tienen alineación con la secuencia de referencia. El valor "parte X + Y" significa X contigs totalmente desalineados más Y contigs parcialmente desalineados. Esta métrica resume # errores no alineados. contigs descritos anteriormente.

**Unaligned length** es la longitud total de todas las regiones no alineadas del ensamblaje (suma de las longitudes de los contigs completamente no alineados y las partes no alineadas de las parcialmente no alineadas). Esta longitud no incluye bases no llamadas (N) en el ensamblaje.

**Genome fraction (%)**  es el porcentaje de bases alineadas en el genoma de referencia. Una base en el genoma de referencia está alineada si hay al menos un cóntigo con al menos una alineación con esta base.

**Duplication ratio** es el número total de bases alineadas en el ensamblaje dividido por el número total de bases alineadas en el genoma de referencia. Si el ensamblaje contiene muchos contigs que cubren las mismas regiones de la referencia, su relación de duplicación puede ser mucho mayor que 1. Esto puede ocurrir debido a la sobreestimación de multiplicidades repetidas y debido a pequeñas superposiciones entre contigs, entre otras razones.

**`#` N's per 100 kbp** es el número promedio de bases no llamadas (N) por 100,000 bases de ensamblado.

**`#` mismatches per 100 kbp** es el número promedio de desajustes por 100 000 bases alineadas en el ensamblaje. Los verdaderos SNP y los errores de secuenciación no se distinguen y se cuentan por igual.

**`#` indels per 100 kbp** es el número promedio de indels por 100 000 bases alineadas en el ensamblado. Varias indels consecutivas de un solo nucleótido se cuentan como una indel.

**`#` genomic features** es el número de características genómicas (genes, CDS, etc.) en el ensamblaje (completo y parcial), según una lista proporcionada por el usuario de posiciones de características genómicas en el genoma de referencia. Una función está "parcialmente cubierta" si el ensamblaje contiene al menos 100 pb de esta función, pero no la totalidad.

**Total aligned length** es el número total de bases alineadas en el ensamblaje. Un valor suele ser más pequeño que un valor de longitud total porque algunos de los contigs pueden estar desalineados o parcialmente desalineados.

**Largest alignment** es la longitud de la alineación continua más grande del ensamblaje. Un valor puede ser más pequeño que el valor del contig más grande si el contig más grande está mal ensamblado o parcialmente desalineado.

**NA50, NGA50, NAx, NGAx, LA50, LAx, LGA50, LGAx (A" significa "alineado")** son similares a las métricas correspondientes sin "A", pero en este caso se consideran bloques alineados en lugar de contigs. Los bloques alineados se obtienen rompiendo contigs en eventos de desmontaje y eliminando todas las bases no alineadas.

#### 2. Informe resumido- descripción del gráfico (basado en icarus.html)

[Icarus](https://quast.sourceforge.net/icarus) genera una visualización de tamaño de contig y una o más visualizaciones de alineación de contig (si se proporciona el genoma o los genomas de referencia). Todos ellos se encuentran en la carpeta de salida "quast_ 17150_4#79_output/icarus_viewers/". Los enlaces a las visualizaciones y otra información auxiliar se proporcionan en el menú principal de Icarus, que se guarda en "quast_ 17150_4#79_output/icarus.html". Todas las visualizaciones con Icarus contienen una leyenda con una descripción del esquema de colores. Para mover y hacer zoom en la ventana interactiva, puede usar el mouse, los controles de Icarus (panel superior) o los atajos de teclado (+, -, ←, →, use Shift para acelerar la acción).

**Paso 1: abrir icarus.html**

**Paso 2: Haga clic en contener visor de tamaño**

Este tipo de visor dibuja contigs ordenados del más largo al más corto. Este orden es adecuado para comparar solo los contigs más grandes o el número de contigs más largos que un umbral específico. El visor muestra el [N50](https://quast.sourceforge.net/docs/manual.html#N50) y [Nx](https://quast.sourceforge.net/docs/manual.html#Nx) (para el valor x definido por el usuario) con color e indicación textual. Si el genoma de referencia está disponible o al menos se conoce la longitud aproximada del genoma, [NG50](https://quast.sourceforge.net/docs/manual.html#NG50) y [NGx](https://quast.sourceforge.net/docs/manual.html#Nx) también se muestran. También puede atenuar contigs más cortos que un umbral específico utilizando el panel de control de Icarus.

**Picture 12-1-1.png

**Paso 3: haz click en el reporte de QUAST en el menu principal**

**Paso 4: haga clic en el visor de alineación de contig en el menú principal para ver los contig alineados con la secuencia de referencia** 

Este tipo de visor está disponible solo si se proporciona un genoma de referencia. Para genomas grandes (≥ 50 Mbp), cada cromosoma se muestra en un visor independiente. El espectador coloca contigs de acuerdo con su mapeo al genoma de referencia. El espectador también puede visualizar genes, operones y leer la distribución de cobertura a lo largo del genoma, si alguno de ellos se alimentó a QUAST.

**Lecturas adicionales**

https://quast.sourceforge.net/docs/manual.html#sec3.4 

https://github.com/ablab/quast

https://academic.oup.com/bioinformatics/article/29/8/1072/228832


## BONO!

Si está trabajando con BASH en su computadora o en un HPC y tiene demasiados archivos, puede optimizar los comandos, los bucles (for loops) son muy útiles para grandes conjuntos de datos.

Aquí hay una manera de hacerlo.

Cree un nuevo script bash usando nano llamado `assemblies.sh`

In [ ]:
# No ejecute esta celda
# Crea un script para ejecutar el ensamblaje
!nano assemblies.sh

Luego copia y pega el siguiente scrip en su nuevo archivo:

In [ ]:
#!/bin/bash
#Autor: Nathalia Portilla

for i in $(ls *_1.trimmed.fastq.gz); do
NAME=$(basename $i _1.trimmed.fastq.gz)
echo "$NAME"
j="${NAME}_1.trimmed.fastq.gz"
echo "$j"
k="${NAME}_2.trimmed.fastq.gz"
echo "$k"
spades.py -1 $j -2 $k -t 16 --careful --cov-cutoff auto -o ${NAME};
done

Guarda el archivo.

Finalmente, puedes ejecutarlo:

In [ ]:
# No ejecute esta celda
!bash assemblies.sh

___